In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)

In [2]:
# credentials
username = os.environ['DBA_USR']
password = os.environ['DBA_PW']
dsn = '10.220.50.121:1661/ANALYTIC'

In [3]:
# create connection
connection = oracledb.connect(user=username, password=password, dsn=dsn)

In [4]:
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [5]:
# create list of all dates
allDates = ['20241031','20240930','20240829','20240731','20240628','20240531',
            '20240430','20240329','20240229','20240131','20231229','20231130']

In [ ]:
allMLs = []
for date in allDates:
    # print in case of errors
    print(date)

    # open and read the file as a single buffer
    sqlQuery = f'''WITH 
    ML AS (
    SELECT DISTINCT AGREE_ID, GCIF_NO, CIF_NO, NOTE_NO, PRD_TP, 
    PRD_NM,PRD_SEGMENT, ORG_LMT_AMT, CURR_LMT_AMT, BAL, ORIG_INT_RT, CURR_INT_RT, OTH_CURR_INT_RT,
    CLCT_RTNG_FCL, ALLOW_PCT, ALLOW_PCT_ADD, RESTRUCT_CD, PASTDUE_SINCE_DT, PASTDUE_DAYS, WRITE_OFF_AMT,
    WRITE_OFF_DT, WRITE_OFF_LYN  from PDA.MASTER_LOAN
    WHERE BASE_DT = {date}
    AND PRD_NM LIKE '%Credit Card%'
    AND STATUS = '00001'),
    C360 AS (
    SELECT DISTINCT GCIF_NO, SEGMENT, SEGMENT_FUNDING, CUST_NM, OPEN_DATE, 
    LAST_ACTIVE_DATE, CUST_TP, GENDER_CD, AGE, EMPLOYMENT_TYPE, 
    MICRO_SEGMENT, CA_BAL, CA_AVG,
    SA_BAL, SA_AVG, TD_NOA, TD_BAL, TD_AVG,
    SY_FUNDING_BAL, SY_FUNDING_AVG, FUNDING_NOA, FUNDING_BAL,
    FUNDING_AVG, NOA_DORMANT, AUM, CC_PLAFOND, CC_BAL, PL_PLAFOND, 
    PLOAN_BAL, TR_PLAFOND, TR_BAL, PB_PLAFOND, PB_BAL, 
    PRK_PLAFOND, PRK_BAL, PPB_PLAFOND, PPB_BAL, BG_PLAFOND,
    BG_BAL, LC_PLAFOND, LC_BAL, SY_LOAN_PLAFOND, SY_LOAN_BAL,
    KPR_PLAFOND, KPR_BAL, KPM_PLAFOND, KPM_BAL, COLLECT,
    LUM, BA_UL_BAL, BA_ALL_BAL, MF_BAL, BONDS_BAL,TRB,
    M2U_TRX_TOTAL, M2U_TRXAMT_FIN, CRTRX_MTD, CRTRX_AMT_MTD, DBTRX_MTD,
    DBTRX_AMT_MTD
    FROM PDA.CUSTOMER_360
    WHERE BASE_DT = {date}
    )
    SELECT * FROM ML 
    LEFT JOIN C360 
    ON ML.GCIF_NO = C360.GCIF_NO'''

    # run query and store data in df
    ML = pd.read_sql(sqlQuery, engine) 

    # change column names into UPPER CASE
    ML.columns = [x.upper() for x in ML.columns]

    allMLs.append(ML)


20241031
